In [3]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 92kB/s 
     |████████████████████████████████| 3.8MB 35.5MB/s 
     |████████████████████████████████| 450kB 44.5MB/s 
ERROR: tensorflow 1.15.0rc3 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.0 which is incompatible.
ERROR: tensorflow 1.15.0rc3 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1


In [4]:
import tensorflow as tf
tf.__version__

'2.0.0'

Observamos que estamos operando con la versión 2.0.0 de tensorflow, liberada el 30 de septiembre de 2019.

Veamos si tenemos los conocimientos matemáticos necesarios para hacer state-of-the-art deep learning. Esta lección se centrará en lo estrictamente necesario para hacer deep learning.

Definamos de manera sencilla lo que es un grafo. Las redes en deep learning no son más que larguísimas funciones que ante la presencia de decenas de capas se pueden volver terriblemente tediosas de representar. Los grafos nos permiten organizar nuestro pensamiento acerca de las funciones que estamos construyendo, y de hecho mucha investigación actual usa la representación gráfica para desarrollar nuevas y mejores estructuras como lo son las resnet y las efficientnets (más adelante las estudiaremos). Siendo más precisios, las redes neuronales son un tipo particular de grafos, son grafos computacionales. En otras palabras, es más sencillo representar las arquitecturas de nuestras redes por medio de grafos que de por medio de notación matemática; funciones insertadas en funciones insertadas en funciones...

Los grafos computacionales pueden lucir como cualquiera de las siguientes dos arquitecturas, la primera representa la función $(x+y)*z$ y la segunda una arquitectura resnet de 152 capas.

Ahora, para entender las matemáticas necesarias entendamos primero a lo que nos referimos con una red neuronal y en dónde se encuentra cada función que definiremos más adelante.

*Nota: a las redes neuronales también se les llama "artificial neural networks" (ANN) y "perceptrones de múltiples capas".

La siguiente imagen (tomada de la clase CS231n Convolutional Neural Networks for Visual Recognition de Stanford) muestra dos arquitecturas diferentes de red neuronales con capas fully connected (completamente conectadas, en español). En el caso de la izquierda tenemos una red neuronal que cuenta con datos que tienen tres paramétros de entrada. Si nuestros ejemplos son casas, entonces los parámetros pueden ser el número de metros cuadrados que ocupan, la localización y si tienen o no patio). A la primera capa se le llama input layer (capa de entrada, en español). El número de parámetros de la segunda capa es definido por nosotros y a cada parámetro le llamamos unidad computacional o neurona, como se prefiera. En este caso, la arquitectura de la izquierda cuenta con cuatro neuronas. A esta la capa le llamamos hidden layer (capa oculta, español). Finalmente, la output layer (capa de salida) cuenta en este caso con dos neuronas y se utiliza para representar el valor que se le da a cada clase en que podemos clasificar (en el caso de la clasificación de imágenes te indica, por ejemplo, si tu imagen es más probable que sea un gato o un perro; una neurona te da el puntaje con respecto a la clase "perro" y la otra con respecto a la clase "gato".

La arquitectura de la derecha es similar en cuanto que cuenta con capas fully connected pero difiere en que cuenta con dos hidden layer (podemos poner las que queramos, siempre y cuando tomemos en cuenta otros factores con los que lidiaremos más adelante) y con una única neurona en el output layer (podría ser el valor predicho del precio de una acción).

Llamamos a la primera arquitectura como una red neuronal de dos layers y a la segunda como red neuronal de tres layers. Las hidden y output layers definen el número de layers en el nombre de la red.

Dentro de cada neurona suceden, al menos, dos operaciones. Primero, se computa la affine function (función afín, en español): poducto punto entre un vector de pesos y un vector $\vec{x}$. La affine function no es más que una función linear y depende del tipo de red neuronal que queramos construir (por ejemplo, la affine function puede ser una operación de convolución para clasificación de imágenes). Al producto de la affine function sigue una función de no linearidad que suele llevar a los valores negativos a cero. Más adelante entraremos en detalles. Al terminar ambas operaciones decimos que "activamos" la neurona. 

Al considerar todas las neuronas en las hidden layers estamos multiplicando una matriz por otra matriz (tensores de rango dos) 

En esta sesión clasificamos la famosa base de datos MNIST para hacer reconocimiento de números en imágenes. 

In [5]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


Por el momento no hagamos caso a nuestros sets de evaluación. En nuestro set de entrenamiento tenemos 6,000 imágenes de 28 x 28 pixeles. Las imágenes están en blanco y negro por lo que no tienen una dimensión RGB. Cada una de las 6,000 imágenes representa un número entre 0 y 9, 10 número en total, y dicha información se encuentra en `y_train`, de forma 60,000 x 1. Podemos dar `x_train.shape, y_train.shape` para conocer esta información. Notar que los datos están en formato numpy, sin embargo al hacer operaciones de tensorflow 2.0 sobre ellos se convierten automáticamente en tensores.

Aplanamos y convertimos a float nuestras imágenes para que podamos operar sobre ellas.

In [0]:
t = tf.reshape(x_train, (60000,-1))
t = tf.dtypes.cast(t, dtype = tf.float32)

Ahora nuestro tensor `t` tiene la forma `TensorShape([6000, 784])` donde el 784 viene de multiplicar 28 x 28.

Guardemos algunas características relevantes de nuestra imagen. `n` será el número de imágenes que tenemos y `m` nuestro número de parámetros iniciales. Además, nh será el número de neuronas que tendremos en nuestra primera capa, ya hablaremos más tarde de eso.

In [0]:
n, m = t.shape
nh1 = 100

`n` es igual a 60,000, el número de imágenes; m es el número de parámetros de nuestro modelo, 784, un valor por cada pixel.

Introduzcamos una estructura de datos que multiplicará a nuestras imágenes y la llamaremos weights (pesos , en español). Definamos las dimensiones que debe tener nuestro tensor de pesos: 



1.   Nuestros weights harán una multiplación de matrices con nuestras imágenes.
2.   Queremos que tenga en la otra dimensión el número de neuronas que definimos para nuestra primera capa, `nh1`.

Nuestros weights entonces tendrán la forma (m, nh). 



In [0]:
w1 = tf.random.normal((m, nh1))*0.0001 

Finalmente, queremos que nuestros pesos tengan una característica muy particular. En cualquier función existen constantes y variables. En deep learning se suele tratar a nuestros datos (imágenes en este caso) como constantes; el valor de nuestros pixeles no cambiará en ningún momento. Nuestras variables serán nuestros weights; los queremos de forma que sean lo más certeros posibles a la hora de predecir lo que sea que queremos predecir. En otras palabras, nosotros operaremos sobre nuestros pesos, los variaremos hasta que encontremos la combinación de pesos que mejor hagan mapping entre nuestros datos y lo que queremos predecir/clasificar. Específicamente, queremos saber como cambia nuestra función de pérdida cada vez que variemos nuestros pesos.

El método `tf.GradientTape()` nos permite registrar los gradientes de una función con respecto a las variables que la integran. En otras, palabras, cuando activamos `tf.GradientTape()` con respecto a una función estamos indicando a tensorflow que eventualmente vamos a requerir los gradientes de dicha función. 

Quedará más claro conforme vayamos construyendo nuestra red neuronal.

*Nota: el recurso de computar los gradientes es más conocido en la librería de pytorch y su método `autograd`. Para los que prefieran una explicación más técnica: el autograd es un "[...] motor para calcular derivadas (producto jacobiano-vector para ser más precisos). Registra un grafo de todas las operaciones realizadas en un tensor con gradiente habilitado y crea un gráfico acíclico llamado gráfico computacional dinámico. Las hojas de este grafo son tensores de entrada y las raíces son tensores de salida. Los gradientes se calculan trazando el grafo desde la raíz hasta la hoja y multiplicando cada gradiente en el camino usando la regla de la cadena." - [Vaibhav Kumar](https://towardsdatascience.com/pytorch-autograd-understanding-the-heart-of-pytorchs-magic-2686cd94ec95).

Para entender mejor el autograd hagamos lo siguiente. Considera la función $$y(x) = x^2,$$ de nuestra clase de cálculo 1 sabemos que $$y'(x) = 2*x,$$ entonces si $x = 5$, $$y'(5) = 2 * 5 = 10.$$ 

Veamos como autograd nos puede dar el mismo resultado. 

In [0]:
x = tf.Variable(5.)

Definimos nuestra función $y(x) = x^2$ y activamos el `tf.GradientTape()` para requerir los gradientes, en este caso la derivada con respecto a x. 

In [0]:
with tf.GradientTape() as tape:
  y = x**2

Ahora usemos la propiedad `gradient()` de nuestro tensor GradientTape, `tape`, para obtener el gradiente de `y` con respecto a `x` y así para conocer el valor de la derivada. *Nota: usemos numpy para imprimir nuestro resultado.

In [11]:
grad = tape.gradient(y,x)
grad.numpy()

10.0

Nos da como resultado 10. Justo lo que buscabamos. Este mismo procedimiento es el que seguiremos más adelante pero con varias funciones compuestas. Como vemos, nada complicado.

Desarrollemos ahora nuestro "array programming". En la programación científica tenemos una preocupación especial por llevar a acabo operaciones en un gran número de valores de manera paralela. El array programming se ve de manera natural en lenguajes como APL, R y Matlab, y en frameworks para Python como numpy o pytorch. Sin array programming es imposible resolver los problemas computacionales que el deep learning demanda de nosotros, punto.

Dicho esto, ahora dictemos que nuestro tensor `x` no sea de rango cero (escalar) sino de rango uno (vector). Como con el caso anterior, queremos que la misma función $x^2$ se aplique para llegar al resultado de `y` y poesteriormente obtener el gradiente con respecto a nuestros valores en `x`. 

In [12]:
x = tf.Variable([5.,4.,3.,2.,1.,0.])
with tf.GradientTape() as tape:
  y = x**2
tape.gradient(y,x).numpy()

array([10.,  8.,  6.,  4.,  2.,  0.], dtype=float32)

Ahora que estamos más familiarizados con el array programming, introduzcamos dos funciones clave para el deep learning: la función de pérdida y la función ReLU. Ninguna representa mayor dificultad.

Existen muchos tipos diferentes tanto de funciones de pérdida como de funciones de no-linearidad. Aquí utilizamos como pérdida la función de "log softmax": 
$$\operatorname{softmax(x)}_{i} = \frac{e^{x_{i}}}{\sum_{0 \leq j \leq n-1} e^{x_{j}}},$$ o en otra notación:
$$\operatorname{softmax(x)}_{i} = \frac{e^{x_{i}}}{e^{x_{0}} + e^{x_{1}} + \cdots + e^{x_{n-1}}}$$

<!-- donde llamaremos a $Y_i$ como "output" (producto, en español) pues será el producto final de nuestra serie de funciones que representan nuestra red neuronal; y a $\hat{Y_i}$ le llamaremos "target" (objetivo, en español) pues es el valor al que aspiramos que nuestro output sea igual. -->
En la práctia aplicaremos logaritmo a nuestra softmax. No se preocupen por la notación matemática, como dice Jeremy Howard, está forma de escribir es bastante de flojera. Ya veremos como la función requiere una única línea de código gracias al array programming. 

Como función de no linearidad utilizaremos la Rectified Linear Unit (ReLU): 
$$\operatorname{ReLU(x)} = \begin{cases}
    x & \text{si } x > 0, \\
    0 & \text{de otra forma}.
\end{cases}$$

Para ambas funciones $x$ será comunmente un tensor de rango dos y gracias al array programming nuestras funciones podrán ser aplicadas simultáneamente a cada elemento dentro de nuestro tensor.

In [0]:
def log_softmax(x): 
  return x - tf.math.log(tf.math.reduce_sum((tf.math.exp(x)), axis=-1, keepdims=True))

Tensorflow tiene su propia implementación de la función de log softmax, `tf.nn.log_softmax`, y podemos comparar su resultado con el de nuestra `log_softmax` y veremos que el resultado es el mismo.

In [0]:
def relu(x):
  return tf.math.maximum(x, 0.)

De la misma manera, tensorflow cueta con la función `tf.nn.relu` y el resultado es el mismo al de nuestra función `relu`.

Estamos listos para multiplicar nuestros pesos por nuestras imágenes.

In [0]:
t_2 = tf.linalg.matmul(t, w1)

In [16]:
t_2.shape

TensorShape([60000, 100])

Ahora tenemos un tensor con dimensiones `TensorShape([60000, 100])`. Es decir, tenemos un tensor rango 1 por cada una de nuestras 60,000 imágenes y cada imagen ahora tiene propiedades en cada una de nuestras 100 neuronas definidas para nuestra capa 1.

Ahora apliquemos nuestra función `relu` a `t_2`.

In [0]:
t_2_relu = relu(t_2)

Notar que al aplicar una función no estamos alterando la forma de nuestros tensores pues la misma función se aplica a cada elemento del tensor. La forma sigue siendo `TensorShape([60000, 100])`.

In [0]:
t_2_softmax = log_softmax(t_2_relu)

Ahora vemos que una red neuronal no es más que una función integrada por múltiples funciones; funciones compuestas. ¿Dónde está el proceso de entrenamiento? Este proceso se hace gracias al algoritmo llamado "backpropagation". Nos queda para la parte 2.

## Forward pass con layers de tensorflow

Crearemos el forward pass para una red neuronal de dos capas fully connected lineares. Empezaremos desde 0 y construiremos hasta llegar a un modelo Sequential al estilo del famoso upper-level framework de tensorflow, Keras.

### Desde (casi) cero

Comencemos de la misma manera que lo hicimos en la sesión pasada. Definamos cierta información relevante para nuestro modelo.

In [0]:
layer1_units = 100
layer2_units = 50
n, parametros_inic = t.shape

esto quiere decir que nuestra primera capa tendrá 100 neuronas/unidades computacionles y la segunda 50. Nuestros datos tienen 60,000 ejemplos y 784 pixeles por imagen por lo que tenemos el mismo número de parametros

Construyamos con esta información nuestros tensores de weights: `w1` serán los weights de la primera capa y `w2` los de la segunda. Como vimos en nuestra sesión de las matemáticas del deep learning, en la primera capa `w1`, tensor de segundo rango, tendrá que multiplicar a nuestros ejemplos, tensor de segundo rango, en lo que conocemos como una multiplicación de matrices. 

Sabemos que nuetro tensor con las imágenes del MNIST aplanadas tiene la forma `TensorShape([60000, 784])` y queremos que nuestra primera capa tenga 100 neuronas/unidades computacionales, por lo que para realizar la multiplicación de matrices `w1` tendrá forma `TensorShape([784, 100])`. Tenemos entonces $784 * 100 = 78,400$ parámetros entrenables en nuestra primera capa. Otra forma de verlo es que cada neurona de la primera capa tiene 784 parámetros, uno para cada pixel de las imágenes.  Así, el producto entre nuestro tensor con imágenes, `t`, y `w1` tendrá `TensorShape([60,000, 100])`. 

Siguiendo la misma lógica, si queremos que nuestra segunda capa cuente con 50 neuronas, entonces debemos definir weights correspondientes: `w2` tendrá forma `TensorShape([100, 50])`. Por lo que el número de parámetros en la segunda capa será de $100 * 50 = 5,000$. El producto entre la activación neuronal de la primera capa de forma `TensorShape([60,000, 100])` y nuestro `w2` tendrá forma `TensorShape([100, 50])` después de la multiplicación de matrices.  

In [0]:
w1 = tf.random.normal((parametros_inic, layer1_units))*0.0001
w2 = tf.random.normal((layer1_units, layer2_units))*0.0001

In [21]:
x = tf.linalg.matmul(t, w1)
x.shape

TensorShape([60000, 100])

Como esperado, el producto de nuestra multiplicación de matrices en la primera cada tiene forma `TensorShape([60000, 100])`. Ahora aplicamos función de no linearidad ReLU para activar las neuronas de la primera capa.

In [0]:
x = relu(x)

Ahora activemos las neuronas de la segunda capa y apliquemos el softmax para obtener las predicciones que nuestro modelo arroja.

In [0]:
x = tf.linalg.matmul(w1, w2)
x = relu(x)
x = log_softmax(x)

Excelente, ¡tenemos la arquitectura y el forward pass de nuestra red neuronal de dos capas! Y lo hicimos (casi) desde cero. Ahora hagamos algo de "refactoring" para que nuestra arquitectura sea más leíble.

### Desde nuestras capas personalizadas (custom layers)

Vamos a utilizar las herramientas más comunes que utilizan los grandes deep learners. Definamos nuestra capa linear como una subclase de `tensorflow.keras.layers.Layer`, una poderosa herramienta de tensorflow que nos permite definir nuestras capas personalizadas. Cuando definimos nuestra subclase entonces esta debe encapsular un "estado", los weights de la layer, y el paso del tensor inicial al tensor producto. 

In [0]:
from tensorflow.keras.layers import Layer

Al crear nuestra capa definimos la forma que tomarán los pesos de dicha capa. Como cuando creamos nuestros pesos en la subsección pasada, definimos que los pesos tendrán la forma `w_inic = tf.random.normal((parametros_pasados, neuronas))*0.0001` donde `parametros_pasados` es el número de párametos que nuestro `tensor_entrante` tiene cada una de nuestras `units` tendrá que multiplicar. En otras palabras `units` es el número de neuronas de la capa y cada una de ellas interactuará con cada uno de los parámetros de la capa anterior; recordar que estamos tratando con redes neuronales fully connected.

In [0]:
class Linear(Layer):

  def __init__(self, parametros_pasados, neuronas):
      super(Linear, self).__init__()
      
      w_inic = tf.random.normal((parametros_pasados, neuronas))*0.0001
      self.w = tf.Variable(initial_value = w_inic, dtype= "float32", trainable=True)

  def call(self, tensor_entrante):
      return tf.linalg.matmul(tensor_entrante, self.w)

Las instancias de las subclases de `tensorflow.keras.layers.Layer` tienen la propiedad de poder ser llamadas como funciones, miremos:

Creamos nuestra instancia de la subclase Linear que creamos arriba y la nombramos `linear_layer`. Como queremos que represente la primera capa de la red neuronal entonces definimos que los parámetros pasados serán 784, pues tenemos 784 pixeles por cada imagen, y 100 neuronas, pues queremos que la primera capa tenga 100 neuronas. 

*Nota: los nombres de los argumentos de `call` e `__init__` pueden ser diferentes. 

In [0]:
linear_layer = Linear(parametros_pasados=784, neuronas=100)

Excelente, tenemos entonces al correr `linear_layer(t)` podemos observar el mismo resultado que obtuvimos en la subsección pasada al multiplicar `tf.linalg.matmul(t, w1)`. La ventaja es que ahora podemos hacer uso de nuestra subclase `Linear` para otras capas, ya veremos.

Con el método `trainable_variables` de nuestra instancia `linear_layer` (escribir `linear_layer.trainable_variables`) podemos observar nuestras variables, es decir, nuestros weights que serán optimizados más adelante para descubrir nuestra función que traducirá las imágenes a números.

Demos el siguiente paso y creemos un modelo que utilice nuestra layer recién creada, Linear. Esta vez crearemos una subclase de la clase `tensorflow.keras.Model` y la llamaremos trivialmente `TwoLinearNeuronsAndSoftmax`. Queremos crear nuestra red neuronal de dos capas, exactamente igual a la que creamos arriba paso a paso. 

Las subclases de la clase Model nos permiten definir, con método `call` el forward pass, es decir el paso que llevara nuestro tensor de entrada hasta su fase final. En otras palabras, definimos el grafo que se seguirá. En la `__init__` inicializamos nuestras capas, en este caso utilizamos nuestra creada capa Linear con respectivos argumentos. 

In [0]:
from tensorflow.keras import Model

In [0]:
class TwoLinearNeuronsAndSoftmax(Model):

  def __init__(self, parametros_pasados, layer1_neuronas, layer2_neuronas):
    super(TwoLinearNeuronsAndSoftmax, self).__init__()

    self.linear1 = Linear(parametros_pasados=parametros_pasados, neuronas=layer1_neuronas)
    self.linear2 = Linear(parametros_pasados=layer1_neuronas, neuronas=layer2_neuronas)
    self.softmax = tf.keras.layers.Softmax(axis=-1)


  def call(self, inputs):
    x = self.linear1(inputs)
    x = relu(x)
    x = self.linear2(x)
    x = relu(x)
    x = self.softmax(x)
    return x

Creamos una instancia de nuestro modelo y lo llamamos simplemente `red`. Nuestro modelo comienza con 784 parámetros provenientes de las 784 neuronas que tiene cada imagen; tiene 100 neuronas en la primera layer; y 50 en la segunda layer. 

In [0]:
red = TwoLinearNeuronsAndSoftmax(parametros_pasados=784, layer1_neuronas=100, layer2_neuronas=50)

Utilizando el método `build` que todas las instancias de subclases de la clase `tensorflow.keras.Model` tienen (ya sé, es un poco escamoso pero vale la pena que lo repitamos), podemos entonces activar nuestro modelo. Como argumentos indicamos la forma del tensor que ingresaremos, el tensor `t` con nuestros datos; como tenemos 60,000 imágenes con 784 pixeles cada una entonces la forma que ingreamos es (60000, 784). 

In [0]:
red.build((60000,784))

Una vez construido nuestro modelo `red` podemos utilizar su método `summary` para recibir un resumen de lo que abarca nuestro modelo.

In [31]:
red.summary()

Model: "two_linear_neurons_and_softmax"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_1 (Linear)            multiple                  78400     
_________________________________________________________________
linear_2 (Linear)            multiple                  5000      
_________________________________________________________________
softmax (Softmax)            multiple                  0         
Total params: 83,400
Trainable params: 83,400
Non-trainable params: 0
_________________________________________________________________


Llamamos a nuestro modelo sobre nuestro tensor de imágenes `t`. Y la forma es la que esparabamos (60000, 50).

In [32]:
red(t)

<tf.Tensor: id=145, shape=(60000, 50), dtype=float32, numpy=
array([[0.01999966, 0.01999891, 0.01999891, ..., 0.01999891, 0.01999891,
        0.01999891],
       [0.01999917, 0.01999875, 0.01999954, ..., 0.01999875, 0.01999875,
        0.02000156],
       [0.02000059, 0.01999874, 0.02000078, ..., 0.01999874, 0.0200033 ,
        0.02000029],
       ...,
       [0.02000185, 0.01999897, 0.01999897, ..., 0.01999897, 0.0200012 ,
        0.01999897],
       [0.01999901, 0.01999896, 0.02000101, ..., 0.01999896, 0.01999896,
        0.02000188],
       [0.01999903, 0.0199989 , 0.02000056, ..., 0.0199989 , 0.02000131,
        0.0199989 ]], dtype=float32)>

### Desde Sequential 

In [0]:
red = tf.keras.Sequential([Linear(parametros_pasados=784, neuronas=100),
                           tf.keras.layers.ReLU(),
                           Linear(parametros_pasados=100, neuronas=50),
                           tf.keras.layers.ReLU(),
                           tf.keras.layers.Softmax(axis=-1)])

In [35]:
red.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_3 (Linear)            multiple                  78400     
_________________________________________________________________
re_lu (ReLU)                 multiple                  0         
_________________________________________________________________
linear_4 (Linear)            multiple                  5000      
_________________________________________________________________
re_lu_1 (ReLU)               multiple                  0         
_________________________________________________________________
softmax_1 (Softmax)          multiple                  0         
Total params: 83,400
Trainable params: 83,400
Non-trainable params: 0
_________________________________________________________________


In [34]:
red(t)

<tf.Tensor: id=180, shape=(60000, 50), dtype=float32, numpy=
array([[0.01999853, 0.0199994 , 0.02001062, ..., 0.01999963, 0.01999853,
        0.01999853],
       [0.01999868, 0.02000212, 0.02000568, ..., 0.01999868, 0.01999868,
        0.01999868],
       [0.01999903, 0.02000041, 0.02000166, ..., 0.02000176, 0.01999853,
        0.01999853],
       ...,
       [0.01999856, 0.02000054, 0.02000801, ..., 0.02000514, 0.01999856,
        0.01999856],
       [0.01999891, 0.01999962, 0.02000536, ..., 0.01999993, 0.01999891,
        0.01999891],
       [0.02000081, 0.0199985 , 0.02000759, ..., 0.02000002, 0.02000046,
        0.0199985 ]], dtype=float32)>

## Backward pass

You take the gradient of a tensor to help you figure out what you need to do to minimize error. [Cami Williams de Fcebook](https://medium.com/@cwillycs/committing-to-pytorch-by-someone-who-doesnt-know-a-ton-about-pytorch-fa222253cf2d).

Gradient descent is an algorithm that allows us to minimize error efficiently. The error is determined by our data. We have data that is properly classified and improperly classified. We take the gradient to decrease the number of improperly classified items.